In [318]:
import numpy as np
from scipy.sparse import csc_matrix

# prepare data

row = np.array([0, 2, 2, 0, 1, 2, 0])
col = np.array([0, 0, 1, 2, 2, 2, 0])
data = np.array([1, 2, 3, 4, 5, 6, 5])
sp_A = csc_matrix((data, (row, col)), dtype=float)

In [5]:
from scipy.sparse.linalg import spsolve

b = np.array([4, 5, 6])
x = spsolve(sp_A, b)
print(x)

[0. 0. 1.]


In [314]:
import numpy as np
import trimesh

# read unstructured mesh
uns_mesh: trimesh.Trimesh = trimesh.load_mesh('../../static/unsmesh/N2_modified.obj')
str_mesh = trimesh.Trimesh()

uns_mesh.fill_holes()

def vert_dist(msh: trimesh.Trimesh, vidx1: int, vidx2: int) -> float:
    return np.linalg.norm(
        msh.vertices[vidx1] - msh.vertices[vidx2]
    )

In [315]:
# get mesh inner and boundary vertices
# output:
#   - inn_verts
#   - bnd_verts
#   - bnd_length
import numpy_indexed as npi

bnd_edges = npi.difference(uns_mesh.edges_unique, uns_mesh.face_adjacency_edges)
bnd_verts = np.array([*bnd_edges[0]])
bnd_edges = np.delete(bnd_edges, [0], axis=0)
bnd_length = vert_dist(uns_mesh, *bnd_verts[:2])

success = True
while success:
    success = False
    last = bnd_verts[-1]
    for idx in range(len(bnd_edges)):
        if last == bnd_edges[idx][0]:
            success = True
            last = bnd_edges[idx][1]
        elif last == bnd_edges[idx][1]:
            success = True
            last = bnd_edges[idx][0]
        if success:
            bnd_verts = np.append(bnd_verts, last)
            bnd_edges = np.delete(bnd_edges, [idx], axis=0)
            bnd_length += vert_dist(uns_mesh, *bnd_verts[-2:])
            break

inn_verts = npi.difference(uns_mesh.face_adjacency_edges.flatten(), bnd_verts)

In [316]:
# parameterize bound to Square
# assume Z=0.0 in str_mesh
# output:
#   - str_mesh.vertices (only boundary)

from functools import reduce

_scale = 2 # square edge length

last_v = bnd_verts[0]
accumed = 0.

for bnd_v in bnd_verts[1:]:
    old_ratio = accumed / bnd_length
    accumed += vert_dist(uns_mesh, last_v, bnd_v)
    ratio = accumed / bnd_length
    flag = -reduce(
        lambda x, y: x * (1 if (y - old_ratio) * (y - ratio) > 0 else -y),
        [0.25, 0.5, 0.75],
        1
    )
    ratio = max(ratio, flag)
    vpos = (0., 0.)
    if ratio < 0.25:
        vpos = (-(_scale / 2) + _scale * (ratio / 0.25), -_scale / 2)
    elif ratio < 0.5:
        vpos = (_scale / 2,  -(_scale / 2) + _scale * ((ratio - 0.25) / 0.25))
    elif ratio < 0.75:
        vpos = ((_scale / 2) - _scale * ((ratio - 0.5) / 0.25), _scale / 2)
    else:
        vpos = (-_scale / 2, (_scale / 2) - _scale * ((ratio - 0.75) / 0.25))

    str_mesh.vertices = np.vstack([str_mesh.vertices, np.append(vpos, 0.)])
    last_v = bnd_v

In [320]:
# initial weights
# keep row, col, data
from scipy.sparse import csc_matrix


def vectors_angle(msh: trimesh.Trimesh, mid: int, start: int, end: int) -> float:
    vec1: np.array = msh.vertices[start] - msh.vertices[mid]
    vec2: np.array = msh.vertices[end] - msh.vertices[mid]
    return np.arccos(vec1.dot(vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))

sp_row = []
sp_col = []
sp_data = []

diag = {}

for edge in uns_mesh.face_adjacency_edges:
    adj_list_s = uns_mesh.vertex_neighbors[edge[0]]
    adj_list_b = uns_mesh.vertex_neighbors[edge[1]]
    adj_vts = npi.intersection(adj_list_s, adj_list_b)
    if len(adj_vts) != 2:
        adj_vts = adj_vts[:2]
    # assert len(adj_vts) == 2, 'not a manifold'
    # compute cotangent weight of edge
    ang = vectors_angle(uns_mesh, adj_vts[0], *edge)
    sp_row.append(edge[0])
    sp_col.append(edge[1])
    sp_data.append(-np.cos(ang) / np.sin(ang))
    diag[sp_row] += -sp_data
    diag[sp_col] += -sp_data

sp_weights = csc_matrix((sp_data, (sp_row, sp_col)), dtype=float)
print(diag)

TypeError: unhashable type: 'list'